In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statistics import mean
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
import seaborn as sns

In [94]:
from matplotlib.lines import Line2D
custom = [Line2D([], [], marker='o', color='#57d3db', linestyle='None'),
          Line2D([], [], marker='o', color='#db5f57', linestyle='None')]
font = {'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)

In [95]:
columns = ['alpha1', 'alpha9', 'alpha21', 'alpha26', 'alpha48', 'alpha49', 'target']

In [96]:
df = pd.read_excel("F:\Alabama_Internship\EEG\Deepesh Code\Datasets\Dataset_fractal_full_spectrum.xlsx")
# df = Final_df[columns]

X = df.iloc[:, 1 : -1].values
y = df.iloc[:, -1].values

X = pd.DataFrame(X)
y = pd.DataFrame(y)

# X['y'] = y
# X['label'] = X['y'].apply(lambda i: str(i))

# pca = PCA(n_components=6)
# pca_result = pca.fit_transform(X.values)

tsne = TSNE(n_components=2, verbose=1, perplexity=10, n_iter=1000)
tsne_results = tsne.fit_transform(X.values)

X['tsne-one'] = tsne_results[:,0]
X['tsne-two'] = tsne_results[:,1] 
# X['tsne-three'] = tsne_results[:,2]
# X['tsne-four'] = tsne_results[:,3]
# X['tsne-five'] = tsne_results[:,4] 
# X['tsne-six'] = tsne_results[:,5]

X.head()

[t-SNE] Computing 31 nearest neighbors...
[t-SNE] Indexed 40 samples in 0.000s...
[t-SNE] Computed neighbors for 40 samples in 0.010s...
[t-SNE] Computed conditional probabilities for sample 40 / 40
[t-SNE] Mean sigma: 1.244688
[t-SNE] KL divergence after 250 iterations with early exaggeration: 55.306351
[t-SNE] KL divergence after 1000 iterations: 0.333869


0         1         2         3         4         5         6   
0 -0.901782 -0.830329 -0.980897 -1.210650 -1.224758 -1.419926 -1.074092  \
1 -1.320204 -1.623408 -1.431892 -1.526572 -1.251248 -1.418533 -1.513213   
2 -1.415377 -1.589916 -1.634028 -1.491580 -1.620209 -1.544714 -1.448086   
3 -1.423122 -1.452558 -1.510277 -1.284680 -1.437361 -1.409920 -1.221836   
4 -1.251106 -2.087519 -2.080044 -1.635853 -0.872282 -1.528395 -2.354655   

          7         8         9  ...        56        57        58        59   
0 -1.435657 -1.246885 -1.403916  ... -2.035501 -0.766744 -1.523765 -1.978816  \
1 -1.356850 -1.489413 -1.228186  ... -1.410132 -1.400176 -1.487766 -1.404723   
2 -1.537809 -1.251945 -1.101147  ... -1.513522 -1.379455 -1.508229 -1.539795   
3 -1.057294 -0.721919 -0.696260  ... -1.192596 -1.566609 -1.146985 -1.324853   
4 -1.568369 -1.451747 -1.226837  ... -1.373643 -0.960445 -1.608378 -1.885847   

         60        61        62        63   tsne-one   tsne-two  
0 -0.869867 -0.624911 -0.680300 -0.914486  27.044655 -55.661278  
1 -1.288202 -1.190974 -1.167914 -1.247068  28.144835  69.802048  
2 -1.520027 -1.951767 -1.381643 -1.428457  27.711206  60.164238  
3 -1.238634 -1.284644 -1.124432 -1.378407 -36.616199   8.343498  
4 -1.570633 -1.163434 -1.621390 -1.887396  44.413197  17.613514  

[5 rows x 66 columns]

In [97]:
X['y'] = y
X['label'] = X['y'].apply(lambda i: str(i))
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-one", y="tsne-two",
    hue="y",
    palette=sns.color_palette("hls", 2),
    data=X,
    legend="full",
    alpha=1,
    s=100
)
plt.legend(custom, ['Before eating','After eating'], loc='upper right')
plt.grid()

In [98]:
tsne_columns = ['tsne-one', 'tsne-two']

In [99]:
X = X[tsne_columns]
X.head()

tsne-one   tsne-two
0  27.044655 -55.661278
1  28.144835  69.802048
2  27.711206  60.164238
3 -36.616199   8.343498
4  44.413197  17.613514

In [100]:
Participants_count = len(df.index)//2

LR_scores = []
XGB_scores = []
KNN_scores = []
SVM_scores = []
KSVM_scores = []
NB_scores = []
DTC_scores = []
RFC_scores = []
Model_scores_list = []

For Leave one out cross validation (LOOCV method), drop out the pre and post info related to one participant, train the model using the remaining data. Test the model using the info which was dropped during the training stage.

In [101]:
print(Participants_count)

20


In [102]:
for i in range(Participants_count):

    X_train = X.drop(labels = [i, i + Participants_count], axis=0)
    y_train = y.drop(labels = [i, i + Participants_count], axis=0)
    X_test = X.iloc[[i, i + Participants_count],:] 
    y_test = y.iloc[[i, i + Participants_count],:] 

    if(i==0):
        X_train.to_excel("X_dataframe.xlsx")
        y_train.to_excel("y_dataframe.xlsx")

    # Feature Scaling

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    # Dimensionality Reduction

    # PCA - Principle Component Analysis

    #pca = PCA(n_components = 0.95)
    #X_train = pca.fit_transform(X_train)
    #X_test = pca.transform(X_test)
    #explained_variance = pca.explained_variance_ratio_
    #print(explained_variance)

    # LDA - Linear Discriminant Analysis

    #lda = LDA(n_components = 1)
    #X_train = lda.fit_transform(X_train,y_train)
    #X_test = lda.transform(X_test)

    # ML Models Accuracy Computation

    LR_classifier = LogisticRegression(random_state = 0)
    LR_classifier.fit(X_train, y_train)
    LR_scores.append(LR_classifier.score(X_test,y_test))

    XGB_classifier = XGBClassifier()
    XGB_classifier.fit(X_train, y_train)
    XGB_scores.append(XGB_classifier.score(X_test,y_test))
    #plt.bar(range(len(XGB_classifier.feature_importances_)), XGB_classifier.feature_importances_)
    #plt.show()

    KNN_classifier = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
    KNN_classifier.fit(X_train, y_train)
    KNN_scores.append(KNN_classifier.score(X_test,y_test))

    SVM_classifier = SVC(kernel = 'linear', random_state = 0)
    SVM_classifier.fit(X_train, y_train)
    SVM_scores.append(SVM_classifier.score(X_test,y_test))

    KSVM_classifier = SVC(kernel = 'rbf', random_state = 0)
    KSVM_classifier.fit(X_train, y_train)
    KSVM_scores.append(KSVM_classifier.score(X_test,y_test))

    NB_classifier = GaussianNB()
    NB_classifier.fit(X_train, y_train)
    NB_scores.append(NB_classifier.score(X_test,y_test))

    DTC_classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    DTC_classifier.fit(X_train, y_train)
    DTC_scores.append(DTC_classifier.score(X_test,y_test))

    RFC_classifier = RandomForestClassifier(random_state=0)
    RFC_classifier.fit(X_train, y_train)
    RFC_scores.append(RFC_classifier.score(X_test,y_test))

We print out the mean of the accuracies obtained through each of the ML models

In [103]:
model_str = ["Logistic Regression","XG Boost", "KNN", "SVM", "Kernel SVM", "Naive Bayes", "Decision Trees Classifier", "Random Forest Classifier"]

Model_scores_list.append(LR_scores)
Model_scores_list.append(XGB_scores)
Model_scores_list.append(KNN_scores)
Model_scores_list.append(SVM_scores)
Model_scores_list.append(KSVM_scores)
Model_scores_list.append(NB_scores)
Model_scores_list.append(DTC_scores)
Model_scores_list.append(RFC_scores)

for i in range(len(model_str)):

    print(model_str[i] + " Score : " + str(mean(Model_scores_list[i])))

Logistic Regression Score : 0.5
XG Boost Score : 0.425
KNN Score : 0.575
SVM Score : 0.525
Kernel SVM Score : 0.55
Naive Bayes Score : 0.425
Decision Trees Classifier Score : 0.45
Random Forest Classifier Score : 0.5
